# Setup

## Types

In [ ]:
from enum import Enum, auto
from typing import Protocol, List, Optional, Dict, Tuple, Any
from datetime import datetime
from dataclasses import dataclass
from backend.core import Node
from backend.core import Task
from backend.core import Warehouse

# Enums
class AgentStatus(Enum):
    """Possible states of an agent."""
    IDLE = "IDLE"
    MOVING = "MOVING"
    WORKING = "WORKING"
    CHARGING = "CHARGING"
    ERROR = "ERROR"

class AgentType(Enum):
    """Types of agents in the warehouse."""
    PICKER = "PICKER"
    TRANSPORTER = "TRANSPORTER"

class NodeType(Enum):
    """Types of nodes in the warehouse."""
    ENTRY = "ENTRY"
    EXIT = "EXIT"
    NORMAL = "NORMAL"
    CENTER = "CENTER"

class TaskType(Enum):
    """Types of tasks in the warehouse."""
    PICK = "PICK"
    PLACE = "PLACE"
    MOVE = "MOVE"
    CHARGE = "CHARGE"

class TaskStatus(Enum):
    """Possible states of a task."""
    PENDING = "PENDING"
    IN_PROGRESS = "IN_PROGRESS"
    COMPLETED = "COMPLETED"
    FAILED = "FAILED"

# Protocols
class INode(Protocol):
    """Protocol defining the interface for a Node."""
    hash: str
    x: float
    y: float
    neighbours: Dict['INode', float]
    heuristic: float
    type: NodeType
    locked: bool
    is_goal: bool

class IAgent(Protocol):
    """Interface for agent objects."""
    agent_id: int
    node: 'Node'
    weight: float
    status: AgentStatus
    goal_state: str
    mixer: Optional['IMixer']
    path: List['Node']
    battery: float
    agent_type: AgentType
    hash_id: str

class IMixer(Protocol):
    """Interface for mixer objects."""
    warehouse: 'Warehouse'
    tasks: List['Task']
    priority_tasks: List['Task']
    agents: Dict[str, IAgent]
    logs: List[str]
    log_file: str

    def log_event(self, event_type: str, message: str, agent: Optional[IAgent] = None, task: Optional['Task'] = None) -> None:
        """Logs an event in the system."""
        pass

class ITask(Protocol):
    """Protocol defining the interface for a Task."""
    hash_id: str
    initial_state: str
    goal_state: str
    job: TaskType
    priority: int

# Schema Classes
@dataclass
class ItemInformation:
    """Schema for storing item information in the warehouse system."""
    item_id: str  # PK
    name: str
    category: str
    box_weight: float
    box_height: float
    box_price: float
    expiry: Optional[datetime]
    counter: int = 0

@dataclass
class ItemShelf:
    """Schema for mapping items to shelves in the warehouse."""
    item_id: str  # FK
    shelf_id: str  # FK
    order_in_shelf: int
    addition_date: datetime
    accessible_nodes: List[str]
    finale: bool = False

@dataclass
class Rack:
    """Schema for storage racks in the warehouse."""
    rack_id: str  # PK
    is_frozen: bool
    current_capacity: float
    start_coords: Tuple[float, float]  # [x,y]
    center_coords: Tuple[float, float]  # [x,y]
    end_coords: Tuple[float, float]    # [x,y]

@dataclass
class Shelf:
    """Schema for shelves within racks."""
    shelf_id: str  # PK
    rack_id: str   # FK -> Rack
    z_level: float
    current_weight: float
    is_locked: bool = False

@dataclass
class FactsTable:
    """Schema for warehouse configuration and constraints."""
    name: str
    location: str
    warehouse_width: float
    warehouse_length: float
    warehouse_height: float
    n_racks: int
    n_shelfs_per_rack: int
    shelfs_max_height: List[float]  # [z1,z2..]
    shelf_max_width: float
    item_length: float

@dataclass
class Transaction:
    """Schema for warehouse transactions."""
    transaction_id: str  # PK
    transaction_type: str
    item_id: str  # FK -> ItemInformation
    quantity: int
    date: datetime 

## Task

In [ ]:
from typing import Optional
import uuid
from core.types import TaskType, ITask

class Task(ITask):
    """Represents a task to be performed by an agent within the warehouse environment.
    
    Attributes:
        hash_id (str): Unique identifier for the task
        initial_state (str): Starting state/location of the task
        goal_state (str): Target state/location for the task
        job (TaskType): Type of job to be performed
        priority (int): Priority level of the task (0 = normal, 1 = high)
    """

    def __init__(self, goal_state: str, job: TaskType, priority: int = 0, initial_state: str = "", hash_id: Optional[str] = None):
        """Initializes a Task instance.

        Args:
            goal_state (str): Description or identifier of the target state/location for the task.
            job (TaskType): The type of job to be performed
            priority (int, optional): The priority level of the task (0 = normal, 1 = high)
            initial_state (str, optional): Description or identifier of the starting state/location
            hash_id (Optional[str], optional): A unique identifier (Primary Key) for the task

        Raises:
            ValueError: If goal_state or job is empty, or if priority is negative
        """
        if not goal_state:
            raise ValueError("Goal state cannot be empty")
        if not job:
            raise ValueError("Job type cannot be empty")
        if priority < 0:
            raise ValueError("Priority cannot be negative")

        self.hash_id: str = hash_id if hash_id is not None else str(uuid.uuid4())
        self.initial_state: str = initial_state
        self.goal_state: str = goal_state
        self.job: TaskType = job
        self.priority: int = priority

    def __str__(self) -> str:
        """Return a string representation of the task.
        
        Returns:
            str: A human-readable string representation of the task
        """
        return f"Task(Job='{self.job.name}', Goal='{self.goal_state}', Priority={self.priority}, ID='{self.hash_id}')"

    def __repr__(self) -> str:
        """Return a detailed string representation for debugging.
        
        Returns:
            str: A detailed string representation of the task
        """
        return (f"Task(hash_id='{self.hash_id}', initial_state='{self.initial_state}', "
                f"goal_state='{self.goal_state}', job='{self.job.name}', priority={self.priority})")

    def __eq__(self, other: object) -> bool:
        """Check if two tasks are equal based on their unique hash_id.
        
        Args:
            other (object): The object to compare with
            
        Returns:
            bool: True if the tasks are equal, False otherwise
        """
        if not isinstance(other, Task):
            return NotImplemented
        return self.hash_id == other.hash_id

    def __hash__(self) -> int:
        """Return a hash based on the task's unique hash_id.
        
        Returns:
            int: A hash value computed from the task's hash_id
        """
        return hash(self.hash_id)

## Nodes

In [ ]:
from typing import Dict, Optional, List
from enum import Enum, auto
from core.types import NodeType, INode, IAgent
from dataclasses import dataclass, field

class NodeType(Enum):
    """Enumeration of possible node types."""
    NORMAL = auto()
    CENTER = auto()

@dataclass
class Node:
    """Represents a node in the warehouse grid.
    
    Attributes:
        x (int): X coordinate
        y (int): Y coordinate
        node_type (NodeType): Type of the node
        name (str): Name of the node (e.g., "A1", "B2")
        neighbours (Dict[Node, float]): Dictionary of neighboring nodes and their distances
        locked_by (Optional[str]): ID of the agent that has locked this node
    """
    x: int
    y: int
    node_type: NodeType
    name: str
    neighbours: Dict['Node', float] = field(default_factory=dict)
    locked_by: Optional[str] = None
    locked: bool = False
    is_goal: bool = False

    def add_neighbor(self, node: 'Node', distance: float = 1.0) -> None:
        """Adds a neighboring node with the given distance."""
        self.neighbours[node] = distance
        node.neighbours[self] = distance

    def is_locked(self) -> bool:
        """Returns whether the node is currently locked by an agent."""
        return self.locked

    def lock(self, agent_id: str) -> bool:
        """Attempts to lock the node for an agent.
        
        Args:
            agent_id (str): ID of the agent attempting to lock the node
            
        Returns:
            bool: True if the node was successfully locked, False otherwise
        """
        if not self.is_locked():
            self.locked = True
            self.locked_by = agent_id
            return True
        return False

    def unlock(self, agent_id: str) -> bool:
        """Attempts to unlock the node for an agent.
        
        Args:
            agent_id (str): ID of the agent attempting to unlock the node
            
        Returns:
            bool: True if the node was successfully unlocked, False otherwise
        """
        if self.locked and self.locked_by == agent_id:
            self.locked = False
            self.locked_by = None
            return True
        return False

    def __hash__(self) -> int:
        """Returns a hash value for the node."""
        return hash((self.x, self.y))

    def __eq__(self, other: object) -> bool:
        """Checks if two nodes are equal."""
        if not isinstance(other, Node):
            return False
        return self.x == other.x and self.y == other.y

    def __str__(self) -> str:
        """Returns a string representation of the node."""
        return f"{self.name} ({self.x}, {self.y})"

    def get_locking_agent(self) -> Optional[str]:
        """Gets the agent that currently has the node locked.

        Returns:
            Optional[str]: The agent that locked the node, or None if the node is not locked
        """
        return self.locked_by

    def set_type(self, node_type: NodeType) -> None:
        """Sets the type of the node.

        Args:
            node_type (NodeType): The new type for the node
        """
        self.node_type = node_type

    def set_goal(self, is_goal: bool = True) -> None:
        """Sets whether this node is a goal node.

        Args:
            is_goal (bool, optional): Whether this is a goal node. Defaults to True.
        """
        self.is_goal = is_goal


## Warehouse 

In [ ]:
from typing import Dict, Optional, List, TYPE_CHECKING
from math import sqrt
from core.node import Node, NodeType
from core.task import Task
from schema.warehouse import FactsTable
from schema.storage import Rack, Shelf
from dataclasses import dataclass, field
import uuid
import json
import os
from core.types import AgentType

if TYPE_CHECKING:
    from backend.core.agent import Agent

@dataclass
class Warehouse:
    """Represents the warehouse environment.
    
    Attributes:
        facts (FactsTable): Warehouse configuration and facts
        nodes (Dict[str, Node]): All nodes in the warehouse, keyed by node name
        racks (Dict[str, Rack]): All racks in the warehouse, keyed by rack ID
        shelves (Dict[str, Shelf]): All shelves in the warehouse, keyed by shelf ID
        agents (Dict[str, Agent]): All agents in the warehouse
        tasks (List[Task]): All tasks in the warehouse
        goal (Optional[Node]): Current goal node for pathfinding
    """
    facts: FactsTable
    nodes: Dict[str, Node] = field(default_factory=dict)
    racks: Dict[str, Rack] = field(default_factory=dict)
    shelves: Dict[str, Shelf] = field(default_factory=dict)
    agents: Dict[str, 'Agent'] = field(default_factory=dict)
    tasks: List[Task] = field(default_factory=list)
    goal: Optional[Node] = None
    
    @classmethod
    def create_default(cls) -> 'Warehouse':
        """Creates a default warehouse configuration for testing/example purposes."""
        # Create facts table
        facts = FactsTable(
            name="Example Warehouse",
            location="Test Location",
            warehouse_width=20.0,
            warehouse_length=30.0,
            warehouse_height=5.0,
            n_racks=4,
            n_shelfs_per_rack=3,
            shelfs_max_height=[1.0, 2.0, 3.0],
            shelf_max_width=2.0,
            item_length=0.5
        )
        
        # Create a 5x5 grid of nodes
        nodes: Dict[str, Node] = {}
        for i in range(5):
            for j in range(5):
                node_hash = f"node_{i}_{j}"
                nodes[node_hash] = Node(float(i), float(j), node_hash, {}, 0.0)
        
        # Connect nodes (up, down, left, right)
        for i in range(5):
            for j in range(5):
                current = nodes[f"node_{i}_{j}"]
                neighbors = {}
                
                # Check all adjacent positions
                for di, dj in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
                    ni, nj = i + di, j + dj
                    if 0 <= ni < 5 and 0 <= nj < 5:
                        neighbor = nodes[f"node_{ni}_{nj}"]
                        neighbors[neighbor] = 1.0
                
                current.neighbours = neighbors
        
        # Create racks
        racks: Dict[str, Rack] = {}
        rack_positions = [(1, 1), (1, 3), (3, 1), (3, 3)]
        for i, (x, y) in enumerate(rack_positions):
            rack_id = f"rack_{i+1}"
            racks[rack_id] = Rack(
                rack_id=rack_id,
                is_frozen=False,
                current_capacity=0.0,
                start_coords=(float(x-0.5), float(y-0.5)),
                center_coords=(float(x), float(y)),
                end_coords=(float(x+0.5), float(y+0.5))
            )
            # Mark the node as a center
            nodes[f"node_{x}_{y}"].set_type(NodeType.CENTER)
        
        # Create shelves
        shelves: Dict[str, Shelf] = {}
        for rack_id, rack in racks.items():
            for level in range(3):
                shelf_id = f"{rack_id}_shelf_{level+1}"
                shelves[shelf_id] = Shelf(
                    shelf_id=shelf_id,
                    rack_id=rack_id,
                    z_level=float(level + 1),
                    current_weight=0.0,
                    is_locked=False
                )
        
        return cls(facts=facts, nodes=nodes, racks=racks, shelves=shelves)
    
    @classmethod
    def load_from_json(cls, json_path: str) -> 'Warehouse':
        """Loads warehouse configuration from a JSON file.
        
        Args:
            json_path (str): Path to the JSON configuration file
            
        Returns:
            Warehouse: A new Warehouse instance with the loaded configuration
            
        Raises:
            ValueError: If the JSON file is invalid or missing required fields
        """
        try:
            with open(json_path, 'r') as f:
                config = json.load(f)
                
            # Create default facts table
            facts = FactsTable(
                name="Warehouse",
                location="Default",
                warehouse_width=20.0,  # meters
                warehouse_length=30.0,  # meters
                warehouse_height=5.0,   # meters
                n_racks=5,             # number of racks
                n_shelfs_per_rack=3,   # shelves per rack
                shelfs_max_height=[1.0, 2.0, 3.0],  # height of each shelf level
                shelf_max_width=2.0,   # meters
                item_length=0.5        # meters
            )
                
            # Create warehouse instance with facts
            warehouse = cls(facts=facts)
            
            # Create nodes
            nodes = {}
            for node_name, node_data in config['nodes'].items():
                # Convert node type string to enum
                try:
                    node_type = NodeType[node_data['type']]
                except KeyError:
                    raise ValueError(f"Invalid node type: {node_data['type']}")
                    
                node = Node(
                    x=node_data['x'],
                    y=node_data['y'],
                    node_type=node_type,
                    name=node_name
                )
                nodes[node_name] = node
                
            # Create connections
            for node1_name, node2_name in config['connections']:
                node1 = nodes[node1_name]
                node2 = nodes[node2_name]
                node1.add_neighbor(node2)
                node2.add_neighbor(node1)
                
            warehouse.nodes = list(nodes.values())
            
            # Create racks and shelves
            for rack_id, rack_data in config['racks'].items():
                center_node = nodes[rack_data['center']]
                rack = Rack(
                    rack_id=rack_id,
                    is_frozen=False,
                    current_capacity=0.0,
                    start_coords=(center_node.x - 0.5, center_node.y - 0.5),
                    center_coords=(center_node.x, center_node.y),
                    end_coords=(center_node.x + 0.5, center_node.y + 0.5)
                )
                warehouse.racks.append(rack)
                
                # Create shelves for the rack
                for shelf_id in rack_data['shelves']:
                    shelf = Shelf(
                        shelf_id=shelf_id,
                        rack_id=rack.rack_id,
                        z_level=len(rack.shelves) + 1,
                        current_weight=0.0,
                        is_locked=False
                    )
                    warehouse.shelves.append(shelf)
                    
            return warehouse
            
        except Exception as e:
            raise ValueError(f"Failed to load warehouse from {json_path}: {str(e)}")
    
    def get_node(self, x: int, y: int) -> Optional[Node]:
        """Returns the node at the given coordinates."""
        for node in self.nodes.values():
            if node.x == x and node.y == y:
                return node
        return None

    def get_node_by_name(self, name: str) -> Optional[Node]:
        """Returns the node with the given name."""
        return self.nodes.get(name)

    def get_rack_at_node(self, node: Node) -> Optional[Rack]:
        """Returns the rack at the given node."""
        for rack in self.racks.values():
            if rack.center_node == node:
                return rack
        return None

    def get_shelves_for_rack(self, rack: Rack) -> List[Shelf]:
        """Returns all shelves in the given rack."""
        return list(rack.shelves.values())

    def get_distance(self, n1: Node, n2: Node) -> float:
        """Calculates Euclidean distance between two nodes.

        Args:
            n1 (Node): First node.
            n2 (Node): Second node.

        Returns:
            float: Euclidean distance between n1 and n2.
        """
        return sqrt((n1.x - n2.x) ** 2 + (n1.y - n2.y) ** 2)

    def get_actions(self, node: Node) -> Dict[str, float]:
        """Returns a dictionary of possible actions (neighboring nodes) from a given node.

        Args:
            node (Node): The node to get actions from.

        Returns:
            Dict[str, float]: Dictionary of neighboring nodes and their distances.
        """
        return {k: v for k, v in node.neighbours.items() if k != node.parent}

    def assign_heuristics(self):
        """Assigns heuristic values to nodes based on the goal location."""
        if not self.goal:
            return
        for node in self.nodes.values():
            node.set_heuristic(self.get_distance(node, self.goal))

    def add_task(self, task: Task):
        """Adds a task to the warehouse task list.

        Args:
            task (Task): The task to add.
        """
        self.tasks.append(task)

    def assign_task(self, agent: 'Agent'):
        """Assigns the highest priority task to an agent.

        Args:
            agent (Agent): The agent to assign the task to.
        """
        if self.tasks:
            # Sort tasks by priority (lower priority value means higher priority)
            self.tasks.sort(key=lambda x: x.priority)
            task = self.tasks.pop(0)  # Get the highest priority task
            agent.set_goal(task.goal_state)
            print(f"Assigned Task {task.job} to Agent {agent}.")
        else:
            print(f"No tasks available for Agent {agent}.")

    def calculate_heuristics(self, goal_node: Node, agent_type: AgentType) -> Dict[str, float]:
        """Calculates heuristic values for all nodes based on the goal and agent type."""
        heuristics = {}
        
        for node_name, node in self.nodes.items():
            # Base heuristic is Manhattan distance
            distance = abs(node.x - goal_node.x) + abs(node.y - goal_node.y)
            
            # Adjust heuristic based on agent type
            if agent_type == AgentType.PICKER:
                # Pickers prefer paths with fewer obstacles
                if node.type == NodeType.CENTER:
                    distance += 2  # Penalize rack centers
                elif node.type == NodeType.ENTRY or node.type == NodeType.EXIT:
                    distance -= 1  # Favor entry/exit points
            elif agent_type == AgentType.TRANSPORTER:
                # Transporters prefer straight paths
                if node.type == NodeType.CENTER:
                    distance += 3  # Strongly penalize rack centers
                elif node.type == NodeType.ENTRY or node.type == NodeType.EXIT:
                    distance -= 2  # Strongly favor entry/exit points
            
            heuristics[node_name] = distance
            
        return heuristics

    def __repr__(self):
        """String representation of the Warehouse map."""
        return f"Warehouse(Agents: {len(self.agents)}, Tasks: {len(self.tasks)})"


## Agents

In [ ]:
from typing import List, Optional
from dataclasses import dataclass, field
import uuid
from core.types import AgentStatus, AgentType, IMixer, IAgent
from core.node import Node
from core.task import Task

@dataclass
class Agent(IAgent):
    """Represents an agent in the warehouse system.
    
    Attributes:
        agent_id (int): Unique identifier for the agent
        node (Node): Current node where the agent is located
        weight (float): Weight of the agent (used for priority)
        status (AgentStatus): Current status of the agent
        goal_state (str): Target state/location for the agent
        mixer (Optional[IMixer]): Reference to the global mixer instance
        path (List[Node]): Current planned path
        battery (float): Current battery level (0-100)
        agent_type (AgentType): Type of agent
    """
    agent_id: int
    node: Node
    weight: float
    status: AgentStatus = AgentStatus.IDLE
    goal_state: str = ""
    mixer: Optional[IMixer] = None
    path: List[Node] = field(default_factory=list)
    battery: float = 100.0
    agent_type: AgentType = AgentType.PICKER
    hash_id: str = field(default_factory=lambda: str(uuid.uuid4()))

    def __post_init__(self):
        """Initializes an Agent with its node and registers with the mixer."""
        if not self.node:
            raise ValueError("Agent must be initialized with a valid node")
            
        # Add initial node to path
        self.path.append(self.node)
        if not self.node.lock(self):
            raise RuntimeError(f"Failed to lock initial node {self.node}")
            
        # Register with the global Mixer if available
        if self.mixer:
            self.mixer.log_event('agent_creation', f"Agent {self.agent_id} created at {self.node}", self)

    def move(self, new_node: Node) -> bool:
        """Moves the agent to a new node and logs the action."""
        if not new_node:
            if self.mixer:
                self.mixer.log_event('movement_failed', "Cannot move to None node", self)
            return False
            
        # Try to lock the new node
        if not new_node.lock(self):
            if self.mixer:
                self.mixer.log_event('movement_failed', f"Failed to move to {new_node} - node locked", self)
            return False
            
        # Unlock the current node
        self.node.unlock(self)
        
        # Update node and path
        self.node = new_node
        self.path.append(new_node)
        
        # Log the movement if mixer is available
        if self.mixer:
            self.mixer.log_event('movement', f"Moved to {new_node}", self)
        return True

    def backtrack(self, steps: int = 1) -> bool:
        """Moves the agent back along its path history.
        
        Args:
            steps (int): Number of steps to backtrack
            
        Returns:
            bool: True if backtrack was successful, False otherwise
        """
        if steps <= 0:
            if self.mixer:
                self.mixer.log_event('backtrack_failed', f"Invalid number of steps: {steps}", self)
            return False
            
        if len(self.path) <= steps:
            if self.mixer:
                self.mixer.log_event('backtrack_failed', f"Not enough path history to backtrack {steps} steps", self)
            return False
            
        # Get the target node to backtrack to
        target_node = self.path[-steps-1]
        
        # Try to lock the target node
        if not target_node.lock(self):
            if self.mixer:
                self.mixer.log_event('backtrack_failed', f"Failed to backtrack to {target_node} - node locked", self)
            return False
            
        # Unlock the current node
        self.node.unlock(self)
        
        # Update node and path history
        self.node = target_node
        self.path = self.path[:-steps]
        
        # Log the backtrack if mixer is available
        if self.mixer:
            self.mixer.log_event('backtrack', f"Backtracked {steps} steps to {target_node}", self)
        return True

    def set_goal(self, goal: str) -> None:
        """Sets a new goal state for the agent."""
        if not goal:
            raise ValueError("Goal cannot be empty")
            
        self.goal_state = goal
        if self.mixer:
            self.mixer.log_event('goal_change', f"Goal set to {goal}", self)

    def complete_task(self, task: Task) -> None:
        """Completes the given task and logs the completion.
        
        Args:
            task (Task): The task to complete
        """
        if not task:
            raise ValueError("Task cannot be None")
            
        if self.mixer:
            self.mixer.log_event('task_completion', f"Completed task: {task.job} to goal {task.goal_state}", self, task)

    def get_last_node(self) -> Optional[Node]:
        """Returns the last node in the path history.
        
        Returns:
            Optional[Node]: The last node in the path history, or None if history is empty
        """
        return self.path[-1] if self.path else None

    def clear_path_history(self) -> None:
        """Clears the path history, keeping only the current node."""
        if self.path:
            current_node = self.path[-1]
            self.path = [current_node]
            
    def update_battery(self, level: float) -> None:
        """Updates the agent's battery level.
        
        Args:
            level (float): New battery level (0-100)
            
        Raises:
            ValueError: If battery level is not between 0 and 100
        """
        if not 0 <= level <= 100:
            raise ValueError("Battery level must be between 0 and 100")
        self.battery = level
        
        if self.mixer and level < 20:
            self.mixer.log_event('battery_low', f"Battery low ({level}%)", self)

    def __str__(self) -> str:
        """Returns a string representation of the agent."""
        return f"Agent({self.agent_id}, {self.status.name}, {self.node})"

## Mixer

In [ ]:
from typing import List, Dict, Optional, Any
from queue import Queue
from core.task import Task
import json
import os
from datetime import datetime
import uuid
from core.agent import Agent
from core.types import IMixer, IAgent, ITask

class Mixer(IMixer):
    """Handles task assignment, prioritization, and monitoring for agents in the warehouse system.
    Also plays the role of Monitor.
    
    Attributes:
        warehouse (Dict[str, Any]): Reference to the warehouse configuration
        tasks (Queue): Regular tasks queue
        priority_tasks (Queue): High-priority tasks queue
        agents (List[Agent]): List of agents in the system
    """
    
    _instance = None
    LOG_BATCH_SIZE = 100  # Number of logs to accumulate before saving
    
    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super(Mixer, cls).__new__(cls)
            cls._instance._initialized = False
        return cls._instance
    
    def __init__(self, warehouse: Any = None, agents: List[IAgent] = None):
        if self._initialized:
            return
            
        self.warehouse = warehouse  # FK
        self.tasks = Queue()  # Regular tasks queue
        self.priority_tasks = Queue()  # High-priority tasks queue
        self.agents = agents or []  # FK
        self.logs = []  # List to store all system logs
        self._log_file_path = os.path.join(os.path.dirname(__file__), '..', 'data', 'logs')
        self._ensure_log_directory()
        self._initialized = True

    def _ensure_log_directory(self) -> None:
        """Ensures the log directory exists."""
        os.makedirs(self._log_file_path, exist_ok=True)

    def _get_log_file_path(self) -> str:
        """Returns the path to the current log file."""
        timestamp = datetime.now().strftime("%Y%m%d")
        return os.path.join(self._log_file_path, f"warehouse_logs_{timestamp}.json")

    def _save_logs(self) -> None:
        """Saves the current logs to a file."""
        if not self.logs:
            return

        log_file = self._get_log_file_path()
        try:
            # Read existing logs if file exists
            existing_logs = []
            if os.path.exists(log_file):
                with open(log_file, 'r') as f:
                    existing_logs = json.load(f)

            # Append new logs
            existing_logs.extend(self.logs)
            
            # Save all logs
            with open(log_file, 'w') as f:
                json.dump(existing_logs, f, indent=2)
            
            # Clear the in-memory logs
            self.logs = []
            
        except Exception as e:
            print(f"Error saving logs: {e}")

    def log_event(self, event_type: str, message: str, agent: Optional[IAgent] = None, task: Optional[ITask] = None) -> None:
        """Logs an event in the system and saves logs if batch size is reached.
        
        Args:
            event_type (str): Type of event (e.g., 'movement', 'task', 'deadlock')
            message (str): Description of the event
            agent (Agent, optional): Agent involved in the event
            task (Task, optional): Task involved in the event
        """
        log_entry = {
            'id': str(uuid.uuid4()),
            'timestamp': datetime.now().isoformat(),
            'type': event_type,
            'message': message,
            'agent_id': agent.hash_id if agent else None,
            'task_id': task.hash_id if task else None
        }
        self.logs.append(log_entry)
        print(f"[{log_entry['timestamp']}] {event_type}: {message}")

        # Save logs if batch size is reached
        if len(self.logs) >= self.LOG_BATCH_SIZE:
            self._save_logs()

    def get_logs(self, event_type: Optional[str] = None, agent_id: Optional[str] = None, task_id: Optional[str] = None) -> List[Dict]:
        """Retrieves logs based on filters.
        
        Args:
            event_type (str, optional): Filter by event type
            agent_id (str, optional): Filter by agent ID
            task_id (str, optional): Filter by task ID
            
        Returns:
            List[dict]: Filtered log entries
        """
        filtered_logs = self.logs
        if event_type:
            filtered_logs = [log for log in filtered_logs if log['type'] == event_type]
        if agent_id:
            filtered_logs = [log for log in filtered_logs if log['agent_id'] == agent_id]
        if task_id:
            filtered_logs = [log for log in filtered_logs if log['task_id'] == task_id]
        return filtered_logs

    def _load_deadlock_table(self) -> dict:
        """Loads the deadlock table from the JSON file."""
        try:
            file_path = os.path.join(os.path.dirname(__file__), '..', 'data', 'deadlock_table.json')
            with open(file_path, 'r') as f:
                return json.load(f)
        except FileNotFoundError:
            print("Warning: deadlock_table.json not found. Using default deadlock handling.")
            return {
                "deadlock_scenarios": {
                    "agent_blocked": {
                        "action": "move_agent_back",
                        "parameters": {"steps": 1}
                    }
                },
                "default_resolution": {
                    "action": "move_agent_back",
                    "parameters": {"steps": 1}
                }
            }

    def _enqueue(self, agent: Agent, task: Task):
        """Helper method to enqueue tasks based on priority."""
        queue = self.priority_tasks if task.priority == 1 else self.tasks
        queue.put({
            'agent': agent,
            'goal': task.goal_state,
            'job': task.job
        })

    def order(self, agent: Agent, task: Task):
        """Orders a task for an agent by enqueuing it into the appropriate queue."""
        self._enqueue(agent, task)

    def assign_task(self, agent: IAgent) -> None:
        """Assigns tasks from the priority queue first, then from the regular queue."""
        task_assigned = False

        if not self.priority_tasks.empty():
            task = self.priority_tasks.get()
            agent.set_goal(task['goal'])
            task_assigned = True
            print(f"Assigned high-priority Task {task['job']} to Agent {agent}.")
        
        if not task_assigned and not self.tasks.empty():
            task = self.tasks.get()
            agent.set_goal(task['goal'])
            print(f"Assigned Task {task['job']} to Agent {agent}.")

    def detect_and_resolve_deadlock(self):
        """Detects deadlock and resolves it using the deadlock table."""
        blocked_agents = [agent for agent in self.agents if agent.state == "blocked"]

        if len(blocked_agents) > 0:
            # Identify deadlock scenario based on blocked agents
            deadlock_type = self.identify_deadlock_type(blocked_agents)
            scenario = self.deadlock_table["deadlock_scenarios"].get(deadlock_type)
            
            if scenario:
                print(f"Deadlock detected: {scenario['description']}")
                print(f"Resolution: {scenario['resolution']}")
                self.resolve_deadlock(deadlock_type, blocked_agents, scenario)
            else:
                # Use default resolution if scenario not found
                default = self.deadlock_table["default_resolution"]
                print(f"Deadlock detected. Using default resolution: {default['action']}")
                self.resolve_deadlock(deadlock_type, blocked_agents, default)

    def identify_deadlock_type(self, blocked_agents: List[Agent]) -> str:
        """Identifies the type of deadlock based on the blocked agents' situation."""
        if len(blocked_agents) == 1:
            return "agent_blocked"
        elif len(blocked_agents) > 1:
            return "agents_blocked"
        else:
            return "path_blocked"

    def resolve_deadlock(self, deadlock_type: str, blocked_agents: List[Agent], scenario: dict):
        """Resolves the deadlock based on the deadlock type and blocked agents."""
        action = scenario["action"]
        params = scenario.get("parameters", {})

        if action == "move_agent_back":
            self.move_agent_back(blocked_agents, params)
        elif action == "switch_agent_positions":
            self.switch_agent_positions(blocked_agents, params)
        elif action == "recalculate_path":
            self.recalculate_path(blocked_agents, params)
        elif action == "break_circular_wait":
            self.break_circular_wait(blocked_agents, params)
        elif action == "release_resources":
            self.release_resources(blocked_agents, params)

    def move_agent_back(self, agents: List[Agent], params: dict):
        """Moves a blocked agent back along its path history."""
        agent = agents[0]
        steps = params.get("steps", 1)
        
        # Try to backtrack the specified number of steps
        success = agent.backtrack(steps)
        if not success:
            print(f"Warning: Could not backtrack {steps} steps for {agent}. Not enough path history.")
            # If we can't backtrack, try to move back one step at a time
            for _ in range(steps):
                if not agent.backtrack(1):
                    break

    def switch_agent_positions(self, agents: List[Agent], params: dict):
        """Switches positions between two blocked agents."""
        max_agents = params.get("max_agents", 2)
        if len(agents) > max_agents:
            agents = agents[:max_agents]
            
        agent1, agent2 = agents
        # Store current positions
        pos1 = agent1.node
        pos2 = agent2.node
        
        # Move agents to each other's positions
        agent1.move(pos2)
        agent2.move(pos1)
        
        print(f"Switched positions between {agent1} and {agent2}.")

    def recalculate_path(self, agents: List[Agent], params: dict):
        """Recalculates the path for blocked agents."""
        algorithm = params.get("algorithm", "A*")
        max_attempts = params.get("max_attempts", 3)
        print(f"Recalculating path for {agents} using {algorithm} (max attempts: {max_attempts})")

    def break_circular_wait(self, agents: List[Agent], params: dict):
        """Breaks circular wait by moving the lowest weight agent back."""
        priority = params.get("priority", "lowest_weight")
        if priority == "lowest_weight":
            agent = min(agents, key=lambda a: a.weight)
            self.move_agent_back([agent], {"steps": 1})
            print(f"Broke circular wait by moving lowest weight agent: {agent}")

    def release_resources(self, agents: List[Agent], params: dict):
        """Releases and reacquires resources for deadlocked agents."""
        timeout = params.get("timeout", 5)
        retry_interval = params.get("retry_interval", 1)
        print(f"Releasing resources for {agents} (timeout: {timeout}s, retry interval: {retry_interval}s)")


# --------------------------------------------------------------------------------------------

# Item Insertion

# --------------------------------------------------------------------------------------------

## Simmulated Annealing

In [ ]:
import random
import math
import csv
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict

class Candidate:
    """Represents a candidate solution with its state, cost, and efficiency metrics"""
    def __init__(self, state, value, efficiency):
        self.state = state
        self.value = value
        self.efficiency = efficiency

class Problem:
    """Defines the warehouse optimization problem with all constraints"""
    def __init__(self, filename):
        self.filename = filename
        self.items = []
        self.box_sizes = {
            'L': {'width': 180, 'bins': 3},
            'M': {'width': 120, 'bins': 2},
            'S': {'width': 60, 'bins': 1}
        }
        self.load_items()
        
        # Warehouse configuration
        self.num_racks = 36
        self.freezer_racks = {1, 2, 3, 4, 5, 6}
        self.normal_racks = set(range(7, 37))
        self.shelf_levels = 3
        self.bins_per_shelf = 5
        self.bin_size = 60
        self.shelf_length = self.bins_per_shelf * self.bin_size
        
        self.weight_limits = {1: 400, 2: 250, 3: 150}
        self.category_conflicts = {
            'food': ['chemicals'],
            'beverages': ['chemicals'],
            'chemicals': ['food', 'beverages'],
            'frozen': [],
            'household goods': []
        }

    def load_items(self):
        with open(self.filename, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                size = row['item_size'][0].upper()
                if size not in self.box_sizes:
                    raise ValueError(f"Invalid item size '{row['item_size']}'")
                
                self.items.append({
                    'id': int(row['item_id']),
                    'name': row['item_name'],
                    'size': size,
                    'category': row['category'].lower(),
                    'weight': float(row['weight']),
                    'frequency': float(row['frequency']),
                    'is_frozen': row['category'].lower() == 'frozen'
                })

    def get_bin_requirements(self, size):
        """
        Get bin requirements for an item size
        
        Args:
            size: Item size ('L', 'M', or 'S')
            
        Returns:
            tuple: (bins_needed, possible_starting_positions)
        """
        bins = self.box_sizes[size]['bins']
        possible_starts = list(range(1, self.bins_per_shelf - bins + 2))
        return bins, possible_starts

    def generate_initial_state(self):
        """Generate initial random state respecting all constraints"""
        state = {}
        shelf_usage = defaultdict(set)  # {(rack, shelf): set of occupied bins}
        
        for item in self.items:
            placed = False
            attempts = 0
            bins_needed, possible_starts = self.get_bin_requirements(item['size'])
            
            while not placed and attempts < 100:
                # Choose appropriate racks
                valid_racks = self.freezer_racks if item['is_frozen'] else self.normal_racks
                rack = random.choice(list(valid_racks))
                shelf = random.randint(1, self.shelf_levels)
                
                # Find available consecutive bins
                available_starts = [
                    s for s in possible_starts
                    if all(b not in shelf_usage[(rack, shelf)] 
                      for b in range(s, s + bins_needed))
                ]
                
                if available_starts:
                    start = random.choice(available_starts)
                    positions = list(range(start, start + bins_needed))
                    
                    # Place the item
                    state[item['id']] = (rack, shelf, positions)
                    for pos in positions:
                        shelf_usage[(rack, shelf)].add(pos)
                    placed = True
                else:
                    attempts += 1
            
            if not placed:
                # Fallback placement (may violate constraints)
                valid_racks = self.freezer_racks if item['is_frozen'] else self.normal_racks
                rack = random.choice(list(valid_racks))
                shelf = random.randint(1, self.shelf_levels)
                start = random.randint(1, self.bins_per_shelf - bins_needed + 1)
                positions = list(range(start, start + bins_needed))
                state[item['id']] = (rack, shelf, positions)
        
        return state

    def calculate_efficiency(self, state):
        """
        Calculate shelf efficiency metrics
        
        Args:
            state: Current solution state
            
        Returns:
            tuple: (perfect_shelves, good_shelves, poor_shelves, utilization)
        """
        shelf_stats = defaultdict(lambda: {'used_bins': set(), 'weight': 0, 'categories': set()})
        
        # Track shelf usage
        for item_id, (rack, shelf, positions) in state.items():
            item = next(i for i in self.items if i['id'] == item_id)
            shelf_stats[(rack, shelf)]['used_bins'].update(positions)
            shelf_stats[(rack, shelf)]['weight'] += item['weight']
            shelf_stats[(rack, shelf)]['categories'].add(item['category'])
        
        perfect = good = poor = 0
        total_used = 0
        
        # Classify each shelf
        for (rack, shelf), stats in shelf_stats.items():
            used_width = len(stats['used_bins']) * self.bin_size
            remaining = self.shelf_length - used_width
            
            if remaining == 0:
                perfect += 1
            elif remaining <= self.bin_size:  # Can fit at least one S
                good += 1
            elif remaining >= (self.shelf_length / 2):  # More than half empty
                poor += 1
            
            total_used += used_width
        
        # Calculate overall utilization
        total_space = self.num_racks * self.shelf_levels * self.shelf_length
        utilization = total_used / total_space
        
        return (perfect, good, poor, utilization)

    def evaluate(self, state):
        """
        Evaluate the cost of a solution state
        
        Args:
            state: Solution state to evaluate
            
        Returns:
            tuple: (total_cost, efficiency_metrics)
        """
        cost = 0
        shelf_stats = defaultdict(lambda: {'used_bins': set(), 'weight': 0, 'categories': set(), 'items': []})
        
        # Check hard constraints
        for item_id, (rack, shelf, positions) in state.items():
            item = next(i for i in self.items if i['id'] == item_id)
            shelf_key = (rack, shelf)
            stats = shelf_stats[shelf_key]
            
            # Track shelf usage
            stats['used_bins'].update(positions)
            stats['weight'] += item['weight']
            stats['categories'].add(item['category'])
            stats['items'].append(item)
            
            # Frozen items must be in freezer racks
            if item['is_frozen'] and rack not in self.freezer_racks:
                cost += 10000
                
            # Check weight limits
            if stats['weight'] > self.weight_limits[shelf]:
                cost += 1000
                
            # Check category conflicts
            for other in stats['items']:
                if other['category'] in self.category_conflicts[item['category']]:
                    cost += 1000
        
        # Calculate soft costs
        for item_id, (rack, shelf, positions) in state.items():
            item = next(i for i in self.items if i['id'] == item_id)
            cost += item['frequency'] * shelf  # Accessibility cost
            cost += item['weight'] * shelf     # Weight distribution cost
        
        # Add efficiency metrics
        perfect, good, poor, utilization = self.calculate_efficiency(state)
        cost -= perfect * 50  # Reward perfect shelves
        cost += poor * 100    # Penalize poor shelves
        if utilization < 0.85:
            cost += 1000 * (0.85 - utilization)  # Utilization penalty
            
        return cost, (perfect, good, poor, utilization)

    def generate_neighbor(self, current_state):
        """Generate a valid neighboring state by moving one item"""
        new_state = current_state.copy()
        item_id = random.choice(list(new_state.keys()))
        item = next(i for i in self.items if i['id'] == item_id)
        
        # Try up to 20 random moves
        for _ in range(20):
            # Choose appropriate racks
            valid_racks = self.freezer_racks if item['is_frozen'] else self.normal_racks
            rack = random.choice(list(valid_racks))
            shelf = random.randint(1, self.shelf_levels)
            
            bins_needed, possible_starts = self.get_bin_requirements(item['size'])
            
            # Get current shelf usage
            used_bins = set()
            for other_id, (r, s, positions) in new_state.items():
                if r == rack and s == shelf:
                    used_bins.update(positions)
            
            # Find available consecutive bins
            available_starts = [
                s for s in possible_starts
                if all(b not in used_bins for b in range(s, s + bins_needed))
            ]
            
            if available_starts:
                start = random.choice(available_starts)
                positions = list(range(start, start + bins_needed))
                new_state[item_id] = (rack, shelf, positions)
                return new_state
        
        return current_state  # Return original if no valid move found


def simulated_annealing(problem, initial_temp, cooling_rate, iterations):
    """
    Perform simulated annealing optimization
    
    Args:
        problem: Problem instance
        initial_temp: Starting temperature
        cooling_rate: Temperature reduction factor
        iterations: Maximum iterations
        
    Returns:
        tuple: (best_state, best_cost, best_eff, cost_history, perfect_history, util_history, movement_log)
    """
    current_state = problem.generate_initial_state()
    current_cost, current_eff = problem.evaluate(current_state)
    best_state = current_state.copy()
    best_cost = current_cost
    best_eff = current_eff
    
    # Progress tracking
    cost_history = [current_cost]
    perfect_history = [current_eff[0]]
    util_history = [current_eff[3]]
    movement_log = []
    
    for i in range(iterations):
        temp = initial_temp * (cooling_rate ** i)
        if temp < 1e-6:
            break
            
        neighbor = problem.generate_neighbor(current_state)
        neighbor_cost, neighbor_eff = problem.evaluate(neighbor)
        
        # Acceptance probability
        if neighbor_cost < current_cost or random.random() < math.exp((current_cost - neighbor_cost)/temp):
            # Log changes
            changed_items = [
                item_id for item_id in current_state
                if current_state[item_id] != neighbor.get(item_id, None)
            ]
            
            for item_id in changed_items:
                item = next(i for i in problem.items if i['id'] == item_id)
                old_pos = current_state[item_id]
                new_pos = neighbor[item_id]
                reason = determine_move_reason(problem, item, old_pos, new_pos, neighbor_cost - current_cost)
                movement_log.append(create_movement_record(item, old_pos, new_pos, reason, neighbor_cost - current_cost, i))
            
            current_state = neighbor
            current_cost = neighbor_cost
            current_eff = neighbor_eff
            
            if neighbor_cost < best_cost:
                best_state = neighbor
                best_cost = neighbor_cost
                best_eff = neighbor_eff
        
        # Record progress
        cost_history.append(current_cost)
        perfect_history.append(current_eff[0])
        util_history.append(current_eff[3])
        
        if i % 100 == 0:
            print(f"Iter {i}: Cost={current_cost}, Perfect={current_eff[0]}, Util={current_eff[3]:.1%}")
    
    # Add initial to final movement records
    initial_state = problem.generate_initial_state()
    for item in problem.items:
        item_id = item['id']
        initial_pos = initial_state[item_id]
        final_pos = best_state.get(item_id, None)
        
        if final_pos and initial_pos != final_pos:
            reason = "Initial optimization"
            cost_impact = best_cost - cost_history[0]
            movement_log.append(create_movement_record(item, initial_pos, final_pos, reason, cost_impact, "Initial"))
    
    return best_state, best_cost, best_eff, cost_history, perfect_history, util_history, movement_log


def create_movement_record(item, old_pos, new_pos, reason, cost_impact, iteration):
    """
    Create a standardized movement record dictionary
    
    Args:
        item: Item being moved
        old_pos: Original position (rack, shelf, positions)
        new_pos: New position (rack, shelf, positions)
        reason: Reason for move
        cost_impact: Cost change from this move
        iteration: Iteration number
        
    Returns:
        dict: Movement record
    """
    return {
        'item_id': item['id'],
        'item_name': item['name'],
        'category': item['category'],
        'size': item['size'],
        'old_rack': old_pos[0],
        'old_shelf': old_pos[1],
        'old_positions': format_positions(old_pos[2]),
        'new_rack': new_pos[0],
        'new_shelf': new_pos[1],
        'new_positions': format_positions(new_pos[2]),
        'reason': reason,
        'cost_impact': cost_impact,
        'iteration': iteration
    }


def format_positions(positions):
    """
    Format position list as human-readable string
    
    Args:
        positions: List of bin positions
        
    Returns:
        str: Formatted position string (e.g., "2-4" for [2,3,4])
    """
    if len(positions) == 1:
        return str(positions[0])
    return f"{positions[0]}-{positions[-1]}"


def determine_move_reason(problem, item, old_pos, new_pos, cost_change):
    """
    Determine the reason for an item movement
    
    Args:
        problem: Problem instance
        item: Item being moved
        old_pos: Original position
        new_pos: New position
        cost_change: Cost impact of move
        
    Returns:
        str: Reason description
    """
    # Frozen items
    if item['is_frozen'] and new_pos[0] in problem.freezer_racks and old_pos[0] not in problem.freezer_racks:
        return "Moved to freezer rack"
    
    # Category conflicts
    # ... [category conflict checking logic] ...
    
    # Weight distribution
    if new_pos[1] < old_pos[1]:  # Moved to lower shelf
        return "Better weight distribution"
    
    # Space utilization
    old_gaps = calculate_gaps(old_pos[2], problem.bins_per_shelf)
    new_gaps = calculate_gaps(new_pos[2], problem.bins_per_shelf)
    if min(new_gaps) > min(old_gaps):
        return "Improved space utilization"
    
    return "General optimization" if cost_change < 0 else "Exploratory move"


def calculate_gaps(positions, total_bins):
    """
    Calculate gaps between occupied bins
    
    Args:
        positions: List of occupied bin positions
        total_bins: Total bins per shelf
        
    Returns:
        list: Sizes of gaps between occupied bins
    """
    occupied = set(positions)
    gaps = []
    current_gap = 0
    
    for bin in range(1, total_bins + 1):
        if bin in occupied:
            if current_gap > 0:
                gaps.append(current_gap)
            current_gap = 0
        else:
            current_gap += 1
    
    if current_gap > 0:
        gaps.append(current_gap)
    
    return gaps if gaps else [0]



def generate_movement_report(initial_state, best_state, problem, filename='item_movements.csv'):
    """Generate CSV with exact format requested"""
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([
            'item_id', 'name', 'category', 'size',
            'initial_position', 'new_position'
        ])
        
        for item in problem.items:
            item_id = item['id']
            old_pos = initial_state.get(item_id, (None, None, []))
            new_pos = best_state.get(item_id, (None, None, []))
            
            if old_pos[0] and new_pos[0]:  # Only include properly placed items
                writer.writerow([
                    item_id,
                    item['name'],
                    item['category'],
                    item['size'],
                    f"({old_pos[0]},{old_pos[1]},{format_positions(old_pos[2])})",
                    f"({new_pos[0]},{new_pos[1]},{format_positions(new_pos[2])})"
                ])

def generate_heatmaps(initial_state, best_state, problem):
    """Generate before/after heatmaps"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    
    # Prepare data
    def prepare_heatmap_data(state):
        heatmap = np.zeros((problem.shelf_levels, problem.num_racks))
        for (rack, shelf, _) in state.values():
            heatmap[shelf-1][rack-1] += 1
        return heatmap
    
    # Initial state heatmap
    im1 = ax1.imshow(prepare_heatmap_data(initial_state), cmap='YlOrRd')
    ax1.set_title('Initial Item Distribution')
    ax1.set_xlabel('Rack Number')
    ax1.set_ylabel('Shelf Level')
    fig.colorbar(im1, ax=ax1)
    
    # Optimized state heatmap
    im2 = ax2.imshow(prepare_heatmap_data(best_state), cmap='YlOrRd')
    ax2.set_title('Optimized Item Distribution')
    ax2.set_xlabel('Rack Number')
    ax2.set_ylabel('Shelf Level')
    fig.colorbar(im2, ax=ax2)
    
    plt.savefig('placement_heatmaps.png')

def format_positioSns(positions):
    """Improved bin formatting"""
    if not positions:
        return ""
    if len(positions) == 1:
        return str(positions[0])
    return f"{positions[0]}-{positions[-1]}"
def generate_visualization(cost_history, perfect_history, util_history):
    """
    Generate optimization progress plots
    
    Args:
        cost_history: List of cost values over iterations
        perfect_history: List of perfect shelf counts
        util_history: List of utilization percentages
    """
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.plot(cost_history)
    plt.title('Cost Reduction Over Time')
    plt.xlabel('Iteration')
    plt.ylabel('Total Cost')
    
    plt.subplot(1, 3, 2)
    plt.plot(perfect_history)
    plt.title('Perfect Shelves Over Time')
    plt.xlabel('Iteration')
    plt.ylabel('Count')
    
    plt.subplot(1, 3, 3)
    plt.plot(util_history)
    plt.title('Space Utilization Over Time')
    plt.xlabel('Iteration')
    plt.ylabel('Utilization %')
    
    plt.tight_layout()
    plt.savefig('optimization_progress.png')


def generate_text_report(problem, best_state, best_cost, best_eff):
    """
    Generate detailed text report of final solution
    
    Args:
        problem: Problem instance
        best_state: Optimal solution state
        best_cost: Solution cost
        best_eff: Efficiency metrics
    """
    shelf_details = defaultdict(list)
    for item_id, (rack, shelf, positions) in best_state.items():
        item = next(i for i in problem.items if i['id'] == item_id)
        shelf_details[(rack, shelf)].append((item, positions))
    
    report = [
        "="*80,
        "WAREHOUSE OPTIMIZATION REPORT",
        "="*80,
        f"\nFinal Statistics:",
        f"- Total Cost: {best_cost}",
        f"- Perfect Shelves: {best_eff[0]} (fully packed)",
        f"- Good Shelves: {best_eff[1]} (<60cm unused)",
        f"- Poor Shelves: {best_eff[2]} (>150cm unused)",
        f"- Space Utilization: {best_eff[3]:.1%}",
        "\n" + "="*80,
        "SHELF DETAILS (POORLY UTILIZED SHELVES):",
        "="*80
    ]
    
    # Show worst shelves
    poor_shelves = sorted(
        [(k, v) for k, v in shelf_details.items()],
        key=lambda x: problem.shelf_length - sum(problem.box_sizes[i['size']]['width'] for i, _ in x[1]),
        reverse=True
    )[:10]  # Top 10 worst
    
    for (rack, shelf), items in poor_shelves:
        used = sum(problem.box_sizes[i['size']]['width'] for i, _ in items)
        report.append(
            f"\nRack {rack} Shelf {shelf} (Used: {used}cm/{problem.shelf_length}cm):"
        )
        for item, positions in items:
            report.append(
                f"  - ID {item['id']}: {item['name']} ({item['size']}, "
                f"{item['weight']}kg, {item['category']}) "
                f"in positions {format_positions(positions)}"
            )
    
    with open('optimization_report.txt', 'w') as f:
        f.write("\n".join(report))



if __name__ == "__main__":
    try:
        # Initialize problem
        problem = Problem('items.csv')
        
        # Get initial state before optimization
        initial_state = problem.generate_initial_state()
        
        # Run optimization
        print("Starting optimization...")
        results = simulated_annealing(
            problem,
            initial_temp=1000,
            cooling_rate=0.995,
            iterations=5000
        )
        best_state, best_cost, best_eff, *_ = results
        
        # Generate outputs
        generate_movement_report(initial_state, best_state, problem)
        generate_heatmaps(initial_state, best_state, problem)
        generate_visualization(*results[3:6])  # Existing progress plots
        generate_text_report(problem, best_state, best_cost, best_eff)
        
        print("\nOptimization complete!")
        print(f"- Final cost: {best_cost}")
        print(f"- Space utilization: {best_eff[3]:.1%}")
        print("- CSV output: item_movements.csv")
        print("- Heatmaps: placement_heatmaps.png")
        print("- Progress plots: optimization_progress.png")
        print("- Report: optimization_report.txt")
    
    except Exception as e:
        print(f"Error occurred: {str(e)}")

# Path Finding

# --------------------------------------------------------------------------------------------

## Greedy

In [ ]:
import json
import heapq
import math
import os
import matplotlib.pyplot as plt

# --- File Paths ---
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
MAP_FILE_PATH = os.path.join(BASE_DIR, "..", "data", "map.json")
LOOKUP_TABLE_PATH = os.path.join(BASE_DIR, "..", "data", "lookup_table.json")

# --- Helper Functions ---
def load_json_file(file_path):
    try:
        with open(file_path, 'r') as f:
            return json.load(f)
    except Exception as e:
        print(f"Error loading JSON from {file_path}: {e}")
        return None

def get_node_coordinates(node_id, graph_nodes):
    node = graph_nodes.get(node_id)
    return (node['x'], node['y']) if node else (None, None)

def heuristic_manhattan(n1, n2, nodes):
    x1, y1 = get_node_coordinates(n1, nodes)
    x2, y2 = get_node_coordinates(n2, nodes)
    return abs(x1 - x2) + abs(y1 - y2) if None not in (x1, y1, x2, y2) else float('inf')

def get_goal_node_from_lookup(item_id, table):
    return table.get(item_id, {}).get("goal_node")

def reconstruct_path(came_from, current):
    path = [current]
    while current in came_from and came_from[current]:
        current = came_from[current]
        path.append(current)
    return path[::-1]

def greedy_search(start, goal, nodes, heuristic):
    open_set = []
    heapq.heappush(open_set, (heuristic(start, goal, nodes), start))
    came_from = {start: None}
    explored = set()

    while open_set:
        _, current = heapq.heappop(open_set)
        if current in explored:
            continue
        explored.add(current)
        if current == goal:
            return reconstruct_path(came_from, current), explored

        for neighbor in nodes[current].get("neighbours", []):
            if neighbor not in explored and not nodes[neighbor].get("locked", False):
                came_from[neighbor] = current
                heapq.heappush(open_set, (heuristic(neighbor, goal, nodes), neighbor))

    return None, explored

def plot_statistics(path_ids, explored, warehouse_nodes, item_id):
    heuristics = [
        heuristic_manhattan(n, path_ids[-1], warehouse_nodes)
        for n in path_ids[:-1]
    ]
    fig, axs = plt.subplots(2, 1, figsize=(10, 6))
    fig.suptitle(f"Greedy Search Haul Statistics for {item_id}")

    axs[0].bar(["Path Length", "Explored Nodes"], [len(path_ids), len(explored)], color=["blue", "orange"])
    axs[0].set_ylabel("Count")
    axs[0].set_title("Path and Explored Nodes")

    axs[1].plot(range(1, len(heuristics) + 1), heuristics, marker='o', color="green")
    axs[1].set_xlabel("Step in Path")
    axs[1].set_ylabel("Heuristic to Goal")
    axs[1].set_title("Heuristic Values Along Path")

    plt.tight_layout()
    plt.show()

# --- Main ---
def main():
    warehouse_map = load_json_file(MAP_FILE_PATH)
    lookup_table = load_json_file(LOOKUP_TABLE_PATH)
    if not warehouse_map or not lookup_table:
        print("Map or lookup table could not be loaded.")
        return

    nodes = warehouse_map["nodes"]
    agent_start_node = "N2-4"  # Start node for the agent, can be modified if needed

    # Iterate over all items in the lookup table
    for item_id, item_data in lookup_table.items():
        goal_node = get_goal_node_from_lookup(item_id, lookup_table)

        if not goal_node or goal_node not in nodes:
            print(f"Invalid goal node for item {item_id}. Skipping...")
            continue

        print(f"Searching from {agent_start_node} to {goal_node} for item {item_id}")
        path, explored = greedy_search(agent_start_node, goal_node, nodes, heuristic_manhattan)

        if path:
            print(f"Path found for {item_id}: {' -> '.join(path)}")
            plot_statistics(path, explored, nodes, item_id)
        else:
            print(f"No path found for {item_id}.")

if __name__ == "__main__":
    main()


## A*

In [ ]:
from typing import List, Optional
from core.node import Node
from math import inf
from data.load_nodes_from_json import load_nodes_from_json 

def a_star_algorithm(start: Node, goal: Node) -> Optional[List[Node]]:
    """Wrapper function for the A* search algorithm.
    
    Args:
        start: The starting node
        goal: The goal node
        
    Returns:
        Optional[List[Node]]: The path from start to goal if one exists, None otherwise
    """
    search = AStarSearch(start, goal)
    return search.search()

class AStarSearch:
    def __init__(self, start: Node, goal: Node):
        self.start = start
        self.goal = goal
        self.open_list = []
        self.closed_list = []
        self.came_from = {}
        self.g_scores = {start: float(0)}  # Store g-scores separately
        self.f_scores = {start: self._heuristic(start)}  # Store f-scores separately

    def search(self) -> Optional[List[Node]]:
        """ Perform the A* search algorithm. """
        self.open_list.append(self.start)
        while self.open_list:
            current_node = self._get_lowest_f_score_node()
            if current_node == self.goal:
                return self._reconstruct_path(current_node)

            self.open_list.remove(current_node)
            self.closed_list.append(current_node)

            for neighbor, cost in current_node.neighbours.items():
                if neighbor in self.closed_list:
                    continue
                tentative_g_score = self.g_scores[current_node] + cost
                if neighbor not in self.open_list:
                    self.open_list.append(neighbor)
                elif tentative_g_score >= self.g_scores.get(neighbor, inf):
                    continue

                self.came_from[neighbor] = current_node
                self.g_scores[neighbor] = tentative_g_score
                self.f_scores[neighbor] = tentative_g_score + self._heuristic(neighbor)

        return None  # No path found

    def _get_lowest_f_score_node(self) -> Node:
        """ Get the node with the lowest f-score (g + h). """
        return min(self.open_list, key=lambda node: self.f_scores.get(node, inf))

    def _reconstruct_path(self, current_node: Node) -> List[Node]:
        """ Reconstruct the path from start to goal. """
        path = [current_node]
        while current_node in self.came_from:
            current_node = self.came_from[current_node]
            path.append(current_node)
        return path[::-1]  # Reverse to get the path from start to goal

    def _heuristic(self, node: Node) -> float:
        """ Calculate the heuristic value for a node. """
        return self.get_distance(node, self.goal)

    def get_distance(self, n1: Node, n2: Node) -> float:
        """ Calculate the Euclidean distance between two nodes. """
        return ((n1.x - n2.x) ** 2 + (n1.y - n2.y) ** 2) ** 0.5


# Example usage:
if __name__ == "__main__":
    # Load nodes from JSON file
    nodes = load_nodes_from_json("backend/data/facts.json","Euclidean")
    
    # Define start and goal nodes (example)
    start_node = nodes["N3-1"]
    goal_node = nodes["N3-24"]
    
    # Run A* algorithm
    path = a_star_algorithm(start_node, goal_node)
    
    if path:
        print("Path found:", [str(node) for node in path])
    else:
        print("No path found.")

# Layout Optimization

## Genetic Algorithm

In [ ]:
import random
import copy
import math

# --- DUMMY DATA AND HELPERS (Assume these are well-defined for your actual data) ---
DUMMY_ITEM_DB = {
    'itemA': {'weight': 15, 'category': 'food', 'slots': 1}, 'itemB': {'weight': 25, 'category': 'food', 'slots': 2},
    'itemC': {'weight': 5, 'category': 'beverages', 'slots': 1}, 'itemD': {'weight': 40, 'category': 'household goods', 'slots': 3},
    'itemE': {'weight': 50, 'category': 'household goods', 'slots': 2}, 'itemF': {'weight': 2, 'category': 'chemicals', 'slots': 1},
    'itemG': {'weight': 8, 'category': 'food', 'slots': 1}, 'itemH': {'weight': 12, 'category': 'beverages', 'slots': 2},
    'itemI': {'weight': 30, 'category': 'household goods', 'slots': 1}, 'itemJ': {'weight': 60, 'category': 'household goods', 'slots': 3},
    'itemK_frozen': {'weight': 10, 'category': 'frozen', 'slots': 1}, 'itemL_frozen': {'weight': 18, 'category': 'frozen', 'slots': 2},
    'itemM_frozen_sml': {'weight': 5, 'category': 'frozen', 'slots': 1}, 'itemN_food_sml': {'weight': 7, 'category': 'food', 'slots': 1},
}
MAX_SLOTS_PER_SHELF = 5
HEAVY_ITEM_THRESHOLD = 20

def get_item_details(item_id):
    return DUMMY_ITEM_DB.get(item_id, {'weight': 0, 'category': 'unknown', 'slots': 1})

COMPATIBILITY_MATRIX = {
    'food': {'food': True, 'beverages': True, 'household goods': True, 'chemicals': False, 'frozen': False},
    'beverages': {'food': True, 'beverages': True, 'household goods': True, 'chemicals': False, 'frozen': False},
    'household goods': {'food': True, 'beverages': True, 'household goods': True, 'chemicals': False, 'frozen': False},
    'chemicals': {'food': False, 'beverages': False, 'household goods': False, 'chemicals': True, 'frozen': False},
    'frozen': {'food': False, 'beverages': False, 'household goods': False, 'chemicals': False, 'frozen': True},
}

def is_compatible_on_shelf(shelf_items_categories, new_item_category, is_rack_frozen):
    if is_rack_frozen and new_item_category != 'frozen':
        return False # Non-frozen item cannot go into a frozen rack's shelf
    if not is_rack_frozen and new_item_category == 'frozen':
        return False # Frozen item cannot go into a non-frozen rack's shelf

    for existing_category in shelf_items_categories:
        if new_item_category == 'frozen' and existing_category != 'frozen': return False
        if existing_category == 'frozen' and new_item_category != 'frozen': return False
        if not COMPATIBILITY_MATRIX.get(existing_category, {}).get(new_item_category, False):
            return False
    return True

def get_shelf_categories(shelf_items):
    return [get_item_details(item_id)['category'] for item_id in shelf_items]

# --- RACK DATA (Placeholder - you'll get this from your Warehouse object) ---
# Assume each rack object/dict has an 'id' and 'is_frozen' attribute
# and a 'layout' attribute: [[L1_items], [L2_items], [L3_items]]
DUMMY_RACKS_DATA = {
    'R01': {'id': 'R01', 'is_frozen': False, 'layout': [['itemA', 'itemC'], ['itemD'], ['itemF', 'itemN_food_sml']]},
    'R02': {'id': 'R02', 'is_frozen': False, 'layout': [['itemJ'], [], ['itemB', 'itemG']]},
    'R03': {'id': 'R03', 'is_frozen': False, 'layout': [[], ['itemE', 'itemH'], []]},
    'R04': {'id': 'R04', 'is_frozen': True, 'layout': [['itemK_frozen'], ['itemL_frozen'], []]},
    'R05': {'id': 'R05', 'is_frozen': True, 'layout': [['itemM_frozen_sml'],[],[]]}
}
# --- Genetic Algorithm Class (Revised for Pool of Racks) ---

class GeneticAlgorithmPoolOptimizer:
    def __init__(self, initial_rack_pool_layouts, is_optimizing_frozen_racks,
                 generations=50, population_size=30, tournament_size=3,
                 crossover_rate=0.8, mutation_rate=0.1, elite_size=2):
        """
        Initializes GA for a pool of racks (either all frozen or all non-frozen).
        Args:
            initial_rack_pool_layouts (list): List of rack layouts.
                Each element is {'id': str, 'is_frozen': bool, 'layout': [[L1], [L2], [L3]]}.
            is_optimizing_frozen_racks (bool): True if this GA instance is for frozen racks.
        """
        if not initial_rack_pool_layouts:
            raise ValueError("Initial rack pool cannot be empty.")

        self.is_optimizing_frozen_racks = is_optimizing_frozen_racks
        self.num_racks_in_pool = len(initial_rack_pool_layouts)

        # Population: list of individuals. Each individual is a list of rack data dicts.
        self.population = []
        for _ in range(population_size):
            # Create a deep copy of the initial pool for each individual in the population
            individual = copy.deepcopy(initial_rack_pool_layouts)
            self.population.append(individual)

        # Introduce initial variations (more advanced mutation might be needed for diversity)
        # For simplicity, we'll rely on the main mutation operator during evolution.
        # Or, apply a light initial shuffle/mutation to each individual if desired.

        self.generations = generations
        self.population_size = population_size
        self.tournament_size = tournament_size
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate # Chance for an individual (pool state) to be mutated
        self.rack_mutation_rate = 0.2 # Chance for a rack within an individual to be part of mutation
        self.shelf_mutation_rate = 0.1 # Chance for a shelf to be chosen for item move/swap
        self.elite_size = elite_size
        self.fitness_cache = {}

        # Fitness weights (these might need different tuning for frozen vs non-frozen)
        self.w_util = 0.40
        self.w_purity = 0.15
        self.w_weight = 0.20
        self.w_compat = 0.20 # Compatibility is handled more by rack type now
        self.w_empty = 0.05

    def _get_shelf_slots_used(self, shelf_items):
        return sum(get_item_details(item_id)['slots'] for item_id in shelf_items)

    def _validate_and_repair_shelf(self, shelf_items, is_rack_frozen):
        validated_items = []
        slots_used = 0
        for item_id in shelf_items:
            item_detail = get_item_details(item_id)
            # Basic check: frozen items only in frozen racks, non-frozen only in non-frozen
            if is_rack_frozen and item_detail['category'] != 'frozen':
                continue # Skip non-frozen item for a frozen rack's shelf
            if not is_rack_frozen and item_detail['category'] == 'frozen':
                continue # Skip frozen item for a non-frozen rack's shelf

            if slots_used + item_detail['slots'] <= MAX_SLOTS_PER_SHELF:
                current_shelf_categories = get_shelf_categories(validated_items)
                if is_compatible_on_shelf(current_shelf_categories, item_detail['category'], is_rack_frozen):
                    validated_items.append(item_id)
                    slots_used += item_detail['slots']
        return validated_items

    def _validate_and_repair_rack_layout(self, rack_data): # rack_data is {'id':.., 'is_frozen':.., 'layout':..}
        is_frozen = rack_data['is_frozen']
        for i in range(len(rack_data['layout'])):
            rack_data['layout'][i] = self._validate_and_repair_shelf(rack_data['layout'][i], is_frozen)
        return rack_data

    def _calculate_fitness_for_single_rack(self, rack_data): # rack_data is {'id':.., 'is_frozen':.., 'layout':..}
        rack_layout = rack_data['layout']
        is_rack_frozen = rack_data['is_frozen']

        total_utilization = 0; total_purity = 0; weight_penalty = 0
        compatibility_penalty_rack = 0; empty_shelf_penalty = 0
        num_shelves = 3

        for level_idx, shelf_items in enumerate(rack_layout):
            slots_used = self._get_shelf_slots_used(shelf_items)
            shelf_utilization = slots_used / MAX_SLOTS_PER_SHELF if MAX_SLOTS_PER_SHELF > 0 else 0
            total_utilization += shelf_utilization

            shelf_item_details = [get_item_details(item_id) for item_id in shelf_items]
            categories_on_shelf = set(details['category'] for details in shelf_item_details)

            for details in shelf_item_details:
                if is_rack_frozen and details['category'] != 'frozen':
                    compatibility_penalty_rack += 50 # Severe penalty for non-frozen in frozen rack
                if not is_rack_frozen and details['category'] == 'frozen':
                    compatibility_penalty_rack += 50 # Severe penalty for frozen in non-frozen rack
                if level_idx > 0 and details['weight'] > HEAVY_ITEM_THRESHOLD:
                    weight_penalty += details['weight'] * level_idx * 0.01 # Scaled penalty

            if categories_on_shelf:
                total_purity += 1.0 / len(categories_on_shelf)
                # Check within-shelf compatibility (already partially handled by validate_and_repair)
                current_shelf_cats = list(categories_on_shelf)
                for i in range(len(current_shelf_cats)):
                    for j in range(i + 1, len(current_shelf_cats)):
                        if not is_compatible_on_shelf([current_shelf_cats[i]], current_shelf_cats[j], is_rack_frozen):
                             compatibility_penalty_rack += 2 # Smaller penalty, as validate should catch most
            else:
                total_purity += 1.0 # Max purity for empty shelf

            is_empty = not shelf_items
            if level_idx > 0 and is_empty and level_idx -1 >=0 and \
               self._get_shelf_slots_used(rack_layout[level_idx-1]) < MAX_SLOTS_PER_SHELF * 0.5:
                empty_shelf_penalty += 0.5
            elif level_idx == 0 and is_empty and any(len(s) > 0 for s in rack_layout[1:]):
                 empty_shelf_penalty += 1

        avg_utilization = total_utilization / num_shelves if num_shelves > 0 else 0
        avg_purity = total_purity / num_shelves if num_shelves > 0 else 0

        fitness = (self.w_util * avg_utilization) + (self.w_purity * avg_purity) - \
                  (self.w_weight * weight_penalty) - (self.w_compat * compatibility_penalty_rack) - \
                  (self.w_empty * empty_shelf_penalty)
        return max(0, fitness)


    def _calculate_overall_fitness(self, individual_pool_state): # individual_pool_state is a list of rack_data dicts
        # For caching, convert the whole state to a hashable form
        # Sorting by rack ID ensures consistent hashing for the same overall state
        sorted_racks_for_hash = sorted(individual_pool_state, key=lambda r: r['id'])
        individual_tuple = tuple(
            (rack['id'], tuple(tuple(sorted(shelf)) for shelf in rack['layout']))
            for rack in sorted_racks_for_hash
        )

        if individual_tuple in self.fitness_cache:
            return self.fitness_cache[individual_tuple]

        total_fitness_sum = 0
        for rack_data in individual_pool_state:
            total_fitness_sum += self._calculate_fitness_for_single_rack(rack_data)
        
        avg_fitness = total_fitness_sum / self.num_racks_in_pool if self.num_racks_in_pool > 0 else 0
        self.fitness_cache[individual_tuple] = avg_fitness
        return avg_fitness

    def _tournament_selection(self):
        # ... (similar to before, but operates on self.population of individuals (pools))
        # Ensure it calls self._calculate_overall_fitness
        if not self.population: raise ValueError("Population empty.")
        tournament_candidates_indices = random.sample(range(len(self.population)), self.tournament_size)
        tournament_candidates = [self.population[i] for i in tournament_candidates_indices]
        
        best_in_tournament = max(tournament_candidates, key=self._calculate_overall_fitness)
        return best_in_tournament


    def _crossover(self, parent1_pool_state, parent2_pool_state):
        # parentX_pool_state is a list of rack_data dicts
        child1_pool_state = copy.deepcopy(parent1_pool_state)
        child2_pool_state = copy.deepcopy(parent2_pool_state)

        if random.random() < self.crossover_rate:
            # Iterate through corresponding racks in the two parent pools
            # Assumes parent1 and parent2 have racks in the same order (e.g., sorted by ID initially)
            # Or, better, ensure racks are identifiable (e.g., by ID) and map them.
            # For simplicity now, assume same order based on initial pool.
            for rack_idx in range(self.num_racks_in_pool):
                # For each shelf level
                for level in range(3): # L1, L2, L3
                    if random.random() < 0.3: # Chance to swap this specific shelf level between these two racks
                        # Ensure rack_idx is valid for both children
                        if rack_idx < len(child1_pool_state) and rack_idx < len(child2_pool_state):
                            # Swap the shelf contents for this level
                            shelf1_content = child1_pool_state[rack_idx]['layout'][level]
                            shelf2_content = child2_pool_state[rack_idx]['layout'][level]
                            child1_pool_state[rack_idx]['layout'][level] = shelf2_content
                            child2_pool_state[rack_idx]['layout'][level] = shelf1_content
            
            # Validate all racks in the new children pools
            for i in range(self.num_racks_in_pool):
                if i < len(child1_pool_state): child1_pool_state[i] = self._validate_and_repair_rack_layout(child1_pool_state[i])
                if i < len(child2_pool_state): child2_pool_state[i] = self._validate_and_repair_rack_layout(child2_pool_state[i])
        
        return child1_pool_state, child2_pool_state

    def _mutate(self, individual_pool_state):
        mutated_pool = copy.deepcopy(individual_pool_state)
        if random.random() > self.mutation_rate: # Chance to mutate this entire pool state
            return mutated_pool # No mutation for this individual

        # Attempt to swap items between shelves of two different racks in the pool
        if self.num_racks_in_pool < 2: # Need at least two racks to swap between
            return mutated_pool

        # Select two different racks from the pool for potential item swap
        rack_idx1, rack_idx2 = random.sample(range(self.num_racks_in_pool), 2)
        
        rack1_data = mutated_pool[rack_idx1]
        rack2_data = mutated_pool[rack_idx2]

        # Ensure racks are of the same type (both frozen or both non-frozen) for swapping
        if rack1_data['is_frozen'] != rack2_data['is_frozen']:
            return mutated_pool # Cannot swap between frozen and non-frozen racks directly here

        # Try a few times to find a valid swap
        for _attempt in range(5): # Max 5 attempts for a valid swap
            # Select a random shelf from each rack
            shelf_level1 = random.randrange(3)
            shelf_level2 = random.randrange(3) # Can be same or different level

            shelf1_items = rack1_data['layout'][shelf_level1]
            shelf2_items = rack2_data['layout'][shelf_level2]

            if not shelf1_items or not shelf2_items: continue # Need items on both shelves

            # Select a random item from each shelf
            item1_idx = random.randrange(len(shelf1_items))
            item2_idx = random.randrange(len(shelf2_items))
            item1_id = shelf1_items[item1_idx]
            item2_id = shelf2_items[item2_idx]

            item1_details = get_item_details(item1_id)
            item2_details = get_item_details(item2_id)

            # Constraint 1: Items must have the same slot size
            if item1_details['slots'] != item2_details['slots']:
                continue

            # Temporarily perform the swap to check validity
            original_shelf1_item = shelf1_items.pop(item1_idx)
            original_shelf2_item = shelf2_items.pop(item2_idx)

            shelf1_items.insert(item1_idx, item2_id) # Item2 goes to shelf1
            shelf2_items.insert(item2_idx, item1_id) # Item1 goes to shelf2

            # Constraint 2 & 3 & 4: Validate both shelves after swap (compatibility, capacity, weight for L1)
            # We simplify validation by just checking the modified shelves.
            # A full re-validation might be safer but more costly.
            
            temp_shelf1_validated = self._validate_and_repair_shelf(list(shelf1_items), rack1_data['is_frozen'])
            temp_shelf2_validated = self._validate_and_repair_shelf(list(shelf2_items), rack2_data['is_frozen'])

            # Check if the swapped items are still present after validation (meaning swap was valid capacity-wise)
            # and that the shelves are still compatible overall.
            valid_swap = True
            if item2_id not in temp_shelf1_validated or item1_id not in temp_shelf2_validated:
                valid_swap = False
            
            # Check overall compatibility of the new shelves
            if valid_swap:
                shelf1_cats = get_shelf_categories(temp_shelf1_validated)
                for cat_idx in range(len(shelf1_cats)): # Check internal compat
                    if not is_compatible_on_shelf(shelf1_cats[:cat_idx], shelf1_cats[cat_idx], rack1_data['is_frozen']):
                        valid_swap = False; break
                if not valid_swap: continue

                shelf2_cats = get_shelf_categories(temp_shelf2_validated)
                for cat_idx in range(len(shelf2_cats)):
                    if not is_compatible_on_shelf(shelf2_cats[:cat_idx], shelf2_cats[cat_idx], rack2_data['is_frozen']):
                        valid_swap = False; break
                if not valid_swap: continue


            # Weight constraint (simplified for this mutation):
            # if shelf_level1 > 0 and item2_details['weight'] > HEAVY_ITEM_THRESHOLD: valid_swap = False
            # if shelf_level2 > 0 and item1_details['weight'] > HEAVY_ITEM_THRESHOLD: valid_swap = False
            # A more robust check would involve the items being replaced.
            # For now, rely on the main fitness function to penalize bad weight distribution over time.

            if valid_swap:
                # print(f"  MUTATION: Swapped {item1_id} (R{rack1_data['id']}-L{shelf_level1+1}) with {item2_id} (R{rack2_data['id']}-L{shelf_level2+1})")
                rack1_data['layout'][shelf_level1] = temp_shelf1_validated
                rack2_data['layout'][shelf_level2] = temp_shelf2_validated
                break # Successful swap, exit attempt loop
            else:
                # Revert swap
                shelf1_items.pop(item1_idx)
                shelf1_items.insert(item1_idx, original_shelf1_item)
                shelf2_items.pop(item2_idx)
                shelf2_items.insert(item2_idx, original_shelf2_item)
        
        # Validate all racks in the pool after potential mutations
        for i in range(self.num_racks_in_pool):
            mutated_pool[i] = self._validate_and_repair_rack_layout(mutated_pool[i])
            
        return mutated_pool

    def run(self):
        self.fitness_cache = {}
        # Initialize best_solution_pool with a deepcopy of the first individual
        best_solution_pool = copy.deepcopy(self.population[0])
        best_overall_fitness = self._calculate_overall_fitness(best_solution_pool)
        print(f"Initial Avg Fitness for {'Frozen' if self.is_optimizing_frozen_racks else 'Non-Frozen'} Pool: {best_overall_fitness:.4f}")

        for generation in range(self.generations):
            # Calculate fitness for all individuals in the population
            pop_with_fitness = []
            for individual_pool in self.population:
                fitness = self._calculate_overall_fitness(individual_pool)
                pop_with_fitness.append((fitness, individual_pool))
            
            pop_with_fitness.sort(key=lambda x: x[0], reverse=True)
            self.population = [item[1] for item in pop_with_fitness] # Update population

            current_gen_best_fitness = pop_with_fitness[0][0]
            if current_gen_best_fitness > best_overall_fitness:
                best_overall_fitness = current_gen_best_fitness
                best_solution_pool = copy.deepcopy(pop_with_fitness[0][1])
                print(f"  Gen {generation}: New Best Avg Pool Fitness = {best_overall_fitness:.4f}")
            elif generation % 5 == 0:
                 print(f"  Gen {generation}: Current Avg Pool Fitness = {best_overall_fitness:.4f}")

            next_generation_population = []
            if self.elite_size > 0:
                next_generation_population.extend(self.population[:self.elite_size])

            while len(next_generation_population) < self.population_size:
                parent1 = self._tournament_selection()
                parent2 = self._tournament_selection()
                child1, child2 = self._crossover(parent1, parent2)
                
                next_generation_population.append(self._mutate(child1))
                if len(next_generation_population) < self.population_size:
                    next_generation_population.append(self._mutate(child2))
            
            self.population = next_generation_population[:self.population_size]
            self.fitness_cache = {} # Clear cache for the next generation

        print(f"Finished GA for {'Frozen' if self.is_optimizing_frozen_racks else 'Non-Frozen'} Pool. Best Avg Fitness: {best_overall_fitness:.4f}")
        return best_solution_pool


# --- Main Orchestration Logic ---
def get_rack_data_from_warehouse(warehouse_object, rack_id):
    # !! CRITICAL PLACEHOLDER !!
    # Replace this with logic to fetch a specific rack's data (id, is_frozen, layout)
    # from your actual 'warehouse_object'.
    # The layout should be [[L1_items], [L2_items], [L3_items]]
    rack_info = DUMMY_RACKS_DATA.get(rack_id)
    if rack_info:
        return copy.deepcopy(rack_info) # Return a copy to avoid modifying dummy data directly
    raise ValueError(f"Rack ID {rack_id} not found in dummy data.")

def get_all_rack_ids_from_warehouse(warehouse_object, frozen_only=None):
    # !! CRITICAL PLACEHOLDER !!
    # Replace this with logic to get all rack IDs from your 'warehouse_object'.
    # If frozen_only is True, return only frozen rack IDs.
    # If frozen_only is False, return only non-frozen rack IDs.
    # If frozen_only is None, return all (not used in this version).
    ids = []
    for r_id, r_data in DUMMY_RACKS_DATA.items():
        if frozen_only is True and r_data['is_frozen']:
            ids.append(r_id)
        elif frozen_only is False and not r_data['is_frozen']:
            ids.append(r_id)
    return ids

def evaluate_overall_efficiency_for_pool(rack_pool_layouts):
    if not rack_pool_layouts: return 0.0
    total_slots_used_pool = 0
    total_possible_slots_pool = 0
    for rack_data in rack_pool_layouts:
        rack_layout = rack_data['layout']
        total_possible_slots_pool += MAX_SLOTS_PER_SHELF * 3
        for shelf_items in rack_layout:
            total_slots_used_pool += sum(get_item_details(item_id)['slots'] for item_id in shelf_items)
    return total_slots_used_pool / total_possible_slots_pool if total_possible_slots_pool > 0 else 0.0


def run_warehouse_reordering_ga(warehouse_object):
    print("--- Starting Warehouse Reordering GA ---")
    all_optimized_racks = {}

    # --- Parameters for GA (can be tuned) ---
    ga_config = {
        "generations": 50,       # Number of generations
        "population_size": 20,   # Number of "entire warehouse states" in population
        "tournament_size": 3,
        "crossover_rate": 0.8,
        "mutation_rate": 0.15,   # Increased mutation for more exploration
        "elite_size": 2
    }

    # 1. Optimize Non-Frozen Racks
    print("\n--- Optimizing NON-FROZEN Rack Pool ---")
    non_frozen_rack_ids = get_all_rack_ids_from_warehouse(warehouse_object, frozen_only=False)
    if non_frozen_rack_ids:
        initial_non_frozen_pool = [get_rack_data_from_warehouse(warehouse_object, r_id) for r_id in non_frozen_rack_ids]
        print(f"Initial Non-Frozen Pool Efficiency: {evaluate_overall_efficiency_for_pool(initial_non_frozen_pool):.2%}")

        non_frozen_ga = GeneticAlgorithmPoolOptimizer(
            initial_rack_pool_layouts=initial_non_frozen_pool,
            is_optimizing_frozen_racks=False,
            **ga_config
        )
        best_non_frozen_pool_state = non_frozen_ga.run()
        for rack_data in best_non_frozen_pool_state:
            all_optimized_racks[rack_data['id']] = rack_data
        print(f"Final Optimized Non-Frozen Pool Efficiency: {evaluate_overall_efficiency_for_pool(best_non_frozen_pool_state):.2%}")
    else:
        print("No non-frozen racks to optimize.")

    # 2. Optimize Frozen Racks
    print("\n--- Optimizing FROZEN Rack Pool ---")
    frozen_rack_ids = get_all_rack_ids_from_warehouse(warehouse_object, frozen_only=True)
    if frozen_rack_ids:
        initial_frozen_pool = [get_rack_data_from_warehouse(warehouse_object, r_id) for r_id in frozen_rack_ids]
        print(f"Initial Frozen Pool Efficiency: {evaluate_overall_efficiency_for_pool(initial_frozen_pool):.2%}")
        
        frozen_ga = GeneticAlgorithmPoolOptimizer(
            initial_rack_pool_layouts=initial_frozen_pool,
            is_optimizing_frozen_racks=True,
            **ga_config # Can use different params for frozen if needed
        )
        best_frozen_pool_state = frozen_ga.run()
        for rack_data in best_frozen_pool_state:
            all_optimized_racks[rack_data['id']] = rack_data
        print(f"Final Optimized Frozen Pool Efficiency: {evaluate_overall_efficiency_for_pool(best_frozen_pool_state):.2%}")
    else:
        print("No frozen racks to optimize.")

    print("\n--- Warehouse Reordering Complete ---")
    return all_optimized_racks


if __name__ == "__main__":
    # This warehouse_object would be an instance of your actual Warehouse class
    # For testing, we pass None and rely on placeholder functions using DUMMY_RACKS_DATA
    dummy_warehouse_object = None 
    
    optimized_warehouse_state = run_warehouse_reordering_ga(dummy_warehouse_object)

    print("\n\n--- Final State of All Optimized Racks ---")
    if optimized_warehouse_state:
        for rack_id, rack_data in optimized_warehouse_state.items():
            is_frz = "(Frozen)" if rack_data['is_frozen'] else "(Non-Frozen)"
            eff = evaluate_overall_efficiency_for_pool([rack_data]) # Efficiency for this single rack
            print(f"Rack ID: {rack_id} {is_frz}, Layout: {rack_data['layout']}, Efficiency: {eff:.2%}")
    else:
        print("No optimization was performed.")